<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/DGRNet%20STEP3-%20Hourly%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns

generator_multiply = 2 #each input record will generate 100 random vectors from the latent space, given the mu and sigma generated by the encoder

#from keras.utils import plot_model
#import matplotlib.pyplot as plt

#window1 = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences_WINDOW-500.npy')
#window2 = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences_WINDOW-1000.npy')
#window = np.concatenate((window1, window2), axis=0)
#train_data = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences-TRAIN.npy')
#test_data = np.load(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences-TEST.npy')



#get data
train_data = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences-TRAIN-Daily-May2024.npy')


#test_data = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences-TEST_hourly.npy')
#all_data = np.concatenate((train_data,test_data),axis=0)
window_label = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences_WINDOW-Daily-May2024.npy')
n_seq = train_data.shape[0]
window_size = train_data.shape[1]
n_features = train_data.shape[2]


plt.suptitle('Sub sequence plotting', fontsize='30')
plt.xlabel('Time', fontsize ='20')
plt.ylabel('Feature 1', fontsize='20')
plt.plot(train_data[:,:,1])
plt.show()

#---------------------------VAE ------------------------------------------
#from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(train_data, window_label, test_size = 0.2, random_state = 42)

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

encoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/VAE_SIMULATION/vae-encoder-saved-round4-latent5-dim256.model')
decoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/VAE_SIMULATION/vae-decoder-saved-round4-latent5-dim256.model')

X_train_encoded = encoder.predict(train_data)
mu, logvar, z = X_train_encoded
sigma = tf.exp(0.5 * logvar)
batch = tf.shape(mu)[0]  #number of recors / batchs
dim = tf.shape(mu)[1] #Ndimension of latent variable
store = list()
storetemp = list()


#For each batch, iterate, get the generator_multipy number of latent vectors with same window_size.
#For each z, concatenate z_mean, so it will become 100 dimensional vector

for i in range(0,batch):
  all_Z_i = tf.random.normal(shape=(generator_multiply,dim), mean = mu[i,:], stddev=sigma[i,:]) #all randorm vectors for this record i
  X_train_decoded = decoder.predict(all_Z_i)
  X_train_decoded = X_train_decoded.reshape((X_train_decoded.shape[0],window_size*n_features))
  a = np.arange(generator_multiply)
  a.fill(window_label[i])
  c=np.concatenate(((X_train_decoded,a[:,None])),axis=1)
  store.append(c)

results1=np.concatenate(store,axis=0)
np.save(r'/content/drive/MyDrive/PHD/2024/labelled_subsquence_data_daily',results1)


#Regression fitting
x=results1[:,:-1]
y=results1[:,window_size*n_features]


from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

#----- get a test set from this data, to avoid further wrangling----------------
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 42)


maxval = x.shape[0]
count_train = int(math.ceil(0.9*maxval))
x_train = x[:count_train]
x_test = x[count_train:]

y_train = y[:count_train]
y_test = y[count_train:]


from sklearn.ensemble import IsolationForest
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(x_train)
# select all rows that are not outliers
mask = yhat != -1
x_train, y_train = x_train[mask, :], y_train[mask]

x_train = x_train.reshape((x_train.shape[0], window_size, n_features))  #DONT RUN IF MLP
x_test = x_test.reshape((x_test.shape[0], window_size, n_features))    #DONT RUN IF MLP

model = Sequential()
model.add(LSTM(128, input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = 16, activation = 'relu'))
#model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])


#reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
es = keras.callbacks.EarlyStopping(patience=20, verbose=1, min_delta=0.0001, monitor='loss', mode='auto', restore_best_weights=True)
n_epochs = 100
#model.fit(x_train, y_train,epochs=5, batch_size=50, verbose=True)

#y_pred = model.predict(x_test)

#transform


history=model.fit( x_train,y_train,
                 epochs=n_epochs,
                 batch_size=50,
                   validation_split=0.1,
                 callbacks=[es])

y_train_pred = model.predict(x_train)
y_pred = model.predict(x_test)

score_train= r2_score(y_train,y_train_pred)
print("r2 score is ==",score_train)
score= r2_score(y_test,y_pred)
print("r2 score is ==",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_pred)))

#plt.scatter(y_test,y_pred);
#plt.xlabel('Actual');
#plt.ylabel('Predicted');
plt.plot(y_test[0:100], color = 'red', label = 'Real data')
plt.plot(y_pred[0:100], color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()



np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/preduber_2.csv',y_pred)
np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/realuber_2.csv',y_test)
print("MAE is==",mean_absolute_error(y_test,y_pred))

#---------------------------------Without VAE------------------------------------

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error



maxval = train_data.shape[0]
count_train = int(math.ceil(0.9*maxval))
x_train = train_data[:count_train]
x_test = train_data[count_train:]

y_train = window_label[:count_train]
y_test = window_label[count_train:]

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1]*n_features))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1]*n_features))

from sklearn.ensemble import IsolationForest
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(x_train)
# select all rows that are not outliers
mask = yhat != -1
x_train, y_train = x_train[mask, :], y_train[mask]


x_train = x_train.reshape((x_train.shape[0], window_size, n_features))  #DONT RUN IF MLP
x_test = x_test.reshape((x_test.shape[0], window_size, n_features))    #DONT RUN IF MLP

from keras.layers import Dense, Activation
from keras.models import Sequential
##!pip uninstall tensorflow
#!pip install tensorflow==2.12.0
#from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

#reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
es = keras.callbacks.EarlyStopping(patience=10, verbose=1, min_delta=0.01, monitor='loss', mode='auto', restore_best_weights=True)
n_epochs = 100

model = Sequential()
model.add(LSTM(512, input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

history=model.fit( x_train,y_train,
                 epochs=n_epochs,
                 batch_size=32,
                   validation_split=0.1,
                 callbacks=[es])


y_pred = model.predict(x_test)


score= r2_score(y_test,y_pred)
print("r2 score is ==",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_pred)))

#plt.scatter(y_test,y_pred);
#plt.xlabel('Actual');
#plt.ylabel('Predicted');
plt.plot(y_test[0:100], color = 'red', label = 'Real data')
plt.plot(y_pred[0:100], color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()


Epoch 1/100
51/51 [==============================] - 0s 8ms/step - loss: 18.9850 - mean_squared_error: 18.9850 - val_loss: 20.4836 - val_mean_squared_error: 20.4836
Epoch 2/100
51/51 [==============================] - 0s 10ms/step - loss: 19.1221 - mean_squared_error: 19.1221 - val_loss: 21.1521 - val_mean_squared_error: 21.1521
Epoch 3/100
51/51 [==============================] - 1s 10ms/step - loss: 19.0547 - mean_squared_error: 19.0547 - val_loss: 20.6740 - val_mean_squared_error: 20.6740
Epoch 4/100
51/51 [==============================] - 0s 8ms/step - loss: 19.3265 - mean_squared_error: 19.3265 - val_loss: 20.4805 - val_mean_squared_error: 20.4805
Epoch 5/100
51/51 [==============================] - 0s 7ms/step - loss: 18.9439 - mean_squared_error: 18.9439 - val_loss: 20.6081 - val_mean_squared_error: 20.6081
Epoch 6/100
51/51 [==============================] - 0s 7ms/step - loss: 19.5127 - mean_squared_error: 19.5127 - val_loss: 20.6994 - val_mean_squared_error: 20.6994
Epoch 7/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')